In [21]:
import torch

In [5]:
import os
os.chdir("/home/mossishahi/helmholtz/cvae")
os.getcwd()

'/home/mossishahi/helmholtz/cvae'

In [2]:
import torch

In [2]:
import scanpy as sc
import numpy as np
from rdkit import Chem
import os
import pandas as pd
import pickle
import joblib
import gc

%load_ext autoreload
%autoreload 2

In [4]:
def save_object(path, obj, type = "pickle"):
  with open(path, "w+b") as f:
    if type == "pickle":
      joblib.dump(obj, path)
    elif type == "anndata":
      sc.write(path, obj)

def canonicalize(smile):
  return Chem.MolToSmiles(Chem.MolFromSmiles(smile))

In [8]:
from coding.modules import gCVAE
from coding.modules import drugCVAE
from coding.modules import dCVAE

In [13]:
cells.X.A.shape

(200, 5000)

In [47]:
input_shape = cells.X.A.shape[1]

# layers_args = {
#     "rna_input_size": 2000,
#     "img_input_dim": 96,
#     "rna_layers": [512, 512],
#     "img_layers": [[64], [128], [256], [512]],
#     "hidden_layers": [256],
#     "img_input_channels": 3,
#     "img_init_block_channels": 64,
#     "latent_dim": 10,
#     "h_dim": 256,
# }
layer_args = {
        "rna_input_size":input_shape,
        "rna_layers":[round(0.8 * input_shape), round(0.6 * input_shape), round(0.4 * input_shape)],
        "hidden_layers": [round(0.4 * input_shape)],
        "latent_dim":round(0.4* drugs.shape[1]),
        "h_dim": round(0.4 * input_shape),
        }


In [48]:
layer_args

{'rna_input_size': 5000,
 'rna_layers': [4000, 3000, 2000],
 'hidden_layers': [2000],
 'latent_dim': 1360,
 'h_dim': 2000}

In [51]:

model = dCVAE.dCVAE(layer_args, num_conditions = drugs.shape[1])


In [52]:
print(model)

dCVAE(
  (rna_encoder): FullyConnected(
    (FC): Sequential(
      (L0): Linear(in_features=8400, out_features=8400, bias=False)
      (B0): BatchNorm1d(8400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (A0): ReLU()
      (D0): Dropout(p=0.2, inplace=False)
      (L1): Linear(in_features=8400, out_features=4000, bias=False)
      (B1): BatchNorm1d(4000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (A1): ReLU()
      (D1): Dropout(p=0.2, inplace=False)
      (L2): Linear(in_features=4000, out_features=3000, bias=False)
      (B2): BatchNorm1d(3000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (A2): ReLU()
      (D2): Dropout(p=0.2, inplace=False)
      (L3): Linear(in_features=3000, out_features=2000, bias=False)
      (B3): BatchNorm1d(2000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (A3): ReLU()
      (D3): Dropout(p=0.2, inplace=False)
      (L4): Linear(in_features=2000, out_featu

In [6]:
drugs = pd.read_parquet("./dataset/grover_base.parquet")

In [19]:
drugs.loc[:, ["latent_1", "latent_2"]].iloc[[1, 2]]

,latent_1,latent_2
O=C(O)CCc1nc(-c2ccccc2)c(-c2ccccc2)o1,0.219819,-0.434457
CNC(=O)Oc1ccc2c(c1)[C@]1(C)CCN(C)[C@@H]1N2C,0.173832,-0.588151


In [10]:
cells = sc.read_h5ad("./dataset/trapnell_cpa.h5ad")

In [11]:
sample_size = 200
cells = cells[:sample_size, :]
indc = np.array([np.where(np.array(drugs.index) == canonicalize(smile)) for smile in cells.obs["SMILES"]]).squeeze(1)[:, -1]
cells.obs[drugs.columns] = drugs.iloc[indc, :].values

Trying to set attribute `.obs` of view, copying.
/home/mossishahi/anaconda3/envs/cvae/lib/python3.7/site-packages/pandas/core/frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


In [22]:
a = torch.tensor([1, 2, 3])
b = torch.tensor([1, 2, 3])

In [24]:
torch.cat((a, b))

tensor([1, 2, 3, 1, 2, 3])

In [12]:
cells.obs.shape

(200, 3460)

In [40]:
batch = {"rna":torch.tensor(cells.X.A),
        "coord":torch.tensor(drugs.iloc[indc, :].values)}

In [53]:
model.forward(batch)

{'recon_rna': [tensor([[0.1412, 0.0000, 0.0000,  ..., 0.0000, 0.0433, 0.0000],
          [0.0149, 0.0000, 0.0000,  ..., 0.0000, 0.0597, 0.2918],
          [0.3296, 0.3335, 0.0000,  ..., 0.0000, 0.4789, 0.0000],
          ...,
          [0.0000, 0.0388, 0.0000,  ..., 0.0090, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.5995,  ..., 0.0834, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
         grad_fn=<ReluBackward0>)],
 'z_rna': tensor([[ 0.5366,  0.5200, -0.1475,  ..., -2.5769, -0.8213,  1.5690],
         [-0.9415, -0.6898, -0.5294,  ..., -1.5224, -0.0803,  0.5265],
         [-2.8616,  0.6577,  2.3165,  ..., -2.0749,  0.4297,  1.0812],
         ...,
         [ 0.0493,  1.2766, -1.6176,  ...,  0.2115, -0.3442,  0.2219],
         [-0.0654, -1.3924,  0.6150,  ..., -0.7794,  0.4253, -0.1874],
         [ 2.1501, -0.6699,  1.0225,  ..., -0.5416, -0.9740,  1.2999]],
        grad_fn=<ViewBackward>)}

In [14]:
cells.write("./dataset/sample_cells.h5ad")

In [16]:
model = drugCVAE.drugCVAE(cells, drugs.columns.tolist(),latent_dim=4000)

['latent_1', 'latent_2', 'latent_3', 'latent_4', 'latent_5', 'latent_6', 'latent_7', 'latent_8', 'latent_9', 'latent_10', 'latent_11', 'latent_12', 'latent_13', 'latent_14', 'latent_15', 'latent_16', 'latent_17', 'latent_18', 'latent_19', 'latent_20', 'latent_21', 'latent_22', 'latent_23', 'latent_24', 'latent_25', 'latent_26', 'latent_27', 'latent_28', 'latent_29', 'latent_30', 'latent_31', 'latent_32', 'latent_33', 'latent_34', 'latent_35', 'latent_36', 'latent_37', 'latent_38', 'latent_39', 'latent_40', 'latent_41', 'latent_42', 'latent_43', 'latent_44', 'latent_45', 'latent_46', 'latent_47', 'latent_48', 'latent_49', 'latent_50', 'latent_51', 'latent_52', 'latent_53', 'latent_54', 'latent_55', 'latent_56', 'latent_57', 'latent_58', 'latent_59', 'latent_60', 'latent_61', 'latent_62', 'latent_63', 'latent_64', 'latent_65', 'latent_66', 'latent_67', 'latent_68', 'latent_69', 'latent_70', 'latent_71', 'latent_72', 'latent_73', 'latent_74', 'latent_75', 'latent_76', 'latent_77', 'latent

In [17]:
model.train()

>>  cpu
Start test
torch.Size([128, 5000])
torch.Size([128, 5000]) shits
here


RuntimeError: size mismatch, m1: [128 x 1600], m2: [5000 x 256] at /opt/conda/conda-bld/pytorch_1579022060824/work/aten/src/TH/generic/THTensorMath.cpp:136